In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [2]:
TRAINING_DIR = '../input/rotten-fruits/dataset/train'
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32, 
                                                      class_mode='categorical',
                                                      target_size=(150, 150))

# **Inception Model**

In [25]:
def create_pre_trained_model():
    inception_model = InceptionV3(input_shape = (150, 150, 3),
                                  include_top = False, 
                                  weights = 'imagenet') 
    inception_model.trainable = False
    return inception_model

In [26]:
inception_model = create_pre_trained_model()
inception_model.summary()

In [27]:
model = tf.keras.models.Sequential([inception_model,                                 
                                    layers.Flatten(),
                                    layers.Dense(1024, activation='relu'),
                                    layers.Dropout(0.4),
                                    layers.Dense(12, activation='softmax')])

In [28]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4),metrics=['accuracy'] )

In [29]:
history = model.fit(train_generator,
                    epochs = 5,
                    verbose = 1)

# **Mobilenet Model**

In [8]:
def create_pre_trained_model():
    mobilenet_model = MobileNetV2(input_shape = (150, 150, 3),
                                  include_top = False, 
                                  weights = 'imagenet') 
    mobilenet_model.trainable = False
    return mobilenet_model

In [9]:
mobilenet_model = create_pre_trained_model()
mobilenet_model.summary()

In [10]:
model = tf.keras.models.Sequential([mobilenet_model,
                                    layers.Dropout(0.4),
                                    layers.Flatten(),
                                    layers.Dense(1024, activation='relu'),
                                    layers.Dense(12, activation='softmax')])

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4),metrics=['accuracy'] )

In [12]:
history = model.fit(train_generator,
                    epochs = 5,
                    verbose = 1)